### Flash PL bitstream onto FPGA

In [ ]:
from pynq import allocate, Overlay;
import numpy as np;
import matplotlib.pyplot as plot;
from PIL import Image as pil;
import cv2;
import math;

np.set_printoptions(suppress=True)

# Laad systeemcomponenten in PL

system = Overlay("bd_wrapper.bit");
dma = system.axi_dma_0;
vpu = system.vpu_accel_0;

# Check if the VPU IP core is the right component by chekcing if the registers exist

for reg in ["in_width", "in_height", "thres"]:
    assert(hasattr(vpu.register_map, reg));

vpu.register_map

### Load input image

In [ ]:
# Laad input image
inp = cv2.imread("edges.ppm", 0)
image = cv2.Canny(inp, 80, 140)

plot.imshow(image, cmap='gray');

### Allocate SDRAM space & copy image to there

In [ ]:
# Wijs SDRAM toe waar wij de afbeelding willen plaatsen

maxlines = 8
in_buffer = allocate(shape=(720, 1280), dtype=np.uint8, cacheable=1)
out_buffer = allocate(shape=(2, maxlines), dtype=np.int32, cacheable=1)
fl_buffer = allocate(shape=(2, maxlines), dtype=np.float32, cacheable=1)

# Kopieer de afbeelding naar deze ruimtet

in_buffer[:] = np.array(image)

### Activate the HLS Image Processing IP Core

In [ ]:
# Geef een commando aan de digitale logica via de S_AXILITE poort

vpu.register_map.in_width = 1280
vpu.register_map.in_height = 720
vpu.register_map.thres = 1

dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)

In [ ]:
vpu.write(0x00,0x81) # write AP_START and AP_AUTOSTART bits

dma.sendchannel.wait()
dma.recvchannel.wait()

In [ ]:
fl_buffer[:] = out_buffer / (1 << 16)

fl_buffer

In [ ]:
# HW results plotted

vis = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
for i in range(0, maxlines):
    rho = fl_buffer[0, i]
    theta = fl_buffer[1, i]
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a*rho
    y0 = b*rho
    wdtby2 = 1280 / 2
    heiby2 = 720 / 2
    pt1 = (int(x0 + 1500*(-b) + (wdtby2)), int(y0 + 1500*(a) + (heiby2)))
    pt2 = (int(x0 - 1500*(-b) + (wdtby2)), int(y0 - 1500*(a) + (heiby2)))
    cv2.line(vis, pt1, pt2, (255,0,0), 3, cv2.LINE_AA)

cv2.imwrite("edges_in.png", image)
cv2.imwrite("edges_out.png", vis)
plot.imshow(vis)

In [ ]:
# SW ref

rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
lines = cv2.HoughLines(image, 1, np.pi / 180, 150, None, 0, 0)

for i in range(0, len(lines)):
    rho = lines[i][0][0]
    theta = lines[i][0][1]
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a * rho
    y0 = b * rho
    pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
    pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
    cv2.line(rgb, pt1, pt2, (255,0,0), 3, cv2.LINE_AA)

plot.imshow(rgb)
lines

In [ ]:
del in_buffer
del out_buffer